In [27]:
import pandas as pd
df=pd.read_csv("D:\study\data\project\cleaned.csv")
lst=[]
for item in df['User_ID'].unique():
    lst2=list(set(df[df['User_ID']==item]['Product_ID']))
    if len(lst2)>0:
        lst.append(lst2)
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te=TransactionEncoder()
te_data=te.fit(lst).transform(lst)
df_x=pd.DataFrame(te_data,columns=te.columns_)
print(df_x)

      P00000142  P00000242  P00000342  P00000442  P00000542  P00000642  \
0          True      False      False      False      False      False   
1         False      False      False      False      False      False   
2         False      False      False      False      False      False   
3         False      False      False      False      False      False   
4         False      False      False      False      False      False   
5          True      False      False      False      False      False   
6         False      False      False      False      False      False   
7         False      False      False       True      False      False   
8         False      False      False      False      False      False   
9         False       True      False      False      False      False   
10        False      False      False      False      False      False   
11        False      False      False      False      False      False   
12        False       True      False 

In [22]:
frequent_items=apriori(df_x,use_colnames=True,min_support=0.04)
print(frequent_items)

       support                           itemsets
0     0.191818                        (P00000142)
1     0.062977                        (P00000242)
2     0.040401                        (P00000342)
3     0.086912                        (P00000642)
4     0.040401                        (P00000742)
5     0.083857                        (P00001042)
6     0.097097                        (P00001142)
7     0.059243                        (P00001642)
8     0.076218                        (P00001742)
9     0.123578                        (P00002142)
10    0.055169                        (P00002442)
11    0.090307                        (P00002542)
12    0.140893                        (P00003242)
13    0.161433                        (P00003442)
14    0.085384                        (P00003642)
15    0.123918                        (P00003942)
16    0.046681                        (P00004542)
17    0.100323                        (P00004742)
18    0.162112                        (P00005042)


In [23]:
rules=association_rules(frequent_items,metric='confidence',min_threshold=0.4)
rules.antecedents=rules.antecedents.apply(lambda x: next(iter(x)))
rules.consequents=rules.consequents.apply(lambda x: next(iter(x)))
rules=rules.sort_values('lift',ascending=False)
print(rules[rules.lift.apply(lambda x:x>=3)])

    antecedents consequents  antecedent support  consequent support   support  \
168   P00127442   P00032042            0.089798            0.092344  0.041928   
169   P00032042   P00127442            0.092344            0.089798  0.041928   
184   P00034042   P00303342            0.097267            0.098625  0.040570   
183   P00303342   P00034042            0.098625            0.097267  0.040570   
742   P00127442   P00125942            0.089798            0.121202  0.045833   
167   P00032042   P00125942            0.092344            0.121202  0.044984   
760   P00127742   P00127642            0.083178            0.126804  0.040231   
761   P00127842   P00127642            0.101171            0.126804  0.045154   
369   P00193542   P00057942            0.102869            0.131387  0.046002   
447   P00100942   P00106042            0.085215            0.151417  0.043456   
868   P00182242   P00182142            0.096418            0.151248  0.048718   
738   P00193542   P00120042 

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import networkx as nx

nx_data=rules[rules.lift>=3]
GA=nx.from_pandas_edgelist(nx_data,source='antecedents',target='consequents',edge_attr='lift')
pos=nx.kamada_kawai_layout(GA,weight='lift')
# pos = nx.nx_agraph.graphviz_layout(GA)
# pos = nx.nx_agraph.graphviz_layout(GA, prog='dot')

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in GA.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in GA.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

for node,adjacencies in enumerate(GA.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = str(adjacencies[0])+' - # of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])
    
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>association_rules graph',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
plot(fig, filename='association_rules.html')

'file://D:\\program\\python\\Scripts\\association_rules.html'

In [30]:
pd.DataFrame(frequent_items).to_csv('frequent_items.csv')
inter_rules=rules[rules.lift.apply(lambda x:x>=3)]
pd.DataFrame(inter_rules).to_csv('inter_rules.csv')